In [30]:
'''
Exploratory data analysis notebook
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
TRAIN_TS_PATH = '/home/noahg/competitive_ml/zindi/2024/special-giggle/data/Train.csv' # train time series csv
TEST_TS_PATH = '/home/noahg/competitive_ml/zindi/2024/special-giggle/data/Test.csv' # test time series csv

In [4]:
train_ts = pd.read_csv(TRAIN_TS_PATH)
test_ts = pd.read_csv(TEST_TS_PATH)

# print some samples of the csv data
print(f'Train time series samples: \n{train_ts.head(n=10)}')
print('*'*45)
print(f'Test time series samples: \n{test_ts.head(n=10)}')
print('*'*45)

Train time series samples: 
              event_id  precipitation  label
0  id_spictby0jfsb_X_0       0.000000      0
1  id_spictby0jfsb_X_1       0.095438      0
2  id_spictby0jfsb_X_2       1.949560      0
3  id_spictby0jfsb_X_3       3.232160      0
4  id_spictby0jfsb_X_4       0.000000      0
5  id_spictby0jfsb_X_5      56.025400      0
6  id_spictby0jfsb_X_6       1.097630      0
7  id_spictby0jfsb_X_7      23.265700      0
8  id_spictby0jfsb_X_8       2.521400      0
9  id_spictby0jfsb_X_9       0.000000      0
*********************************************
Test time series samples: 
              event_id  precipitation
0  id_j7b6sokflo4k_X_0        0.00000
1  id_j7b6sokflo4k_X_1        3.01864
2  id_j7b6sokflo4k_X_2        0.00000
3  id_j7b6sokflo4k_X_3       16.61520
4  id_j7b6sokflo4k_X_4        2.56706
5  id_j7b6sokflo4k_X_5        0.00000
6  id_j7b6sokflo4k_X_6        0.00000
7  id_j7b6sokflo4k_X_7        0.00000
8  id_j7b6sokflo4k_X_8        0.00000
9  id_j7b6sokflo4k_X_9  

In [5]:
'''
Dataset description:

An imbalance dataset is provided to you with some events containing an event and some events do not contain a flood. 
Both sets of events contain randomly select 40-60 weeks prior and post flood date, to ensure the flood event date is not always the middle index.

For each location the CHIRPS precipitation data has been aggregated over a 5km radius.
Composite cloudless images of the areas of interest have been provided along with daily CHIRPS precipitation data from 1981 to 2023.

Data for 2 years, 730 days was collected. Somewhere in these 730 days per location/event ID there could be a flood. 
Due to the sparsity of identified floods we have created an imbalanced dataset where there are some 
"events" or "locations" that do not have a flood in any of the 730 days.

You can think of event_id_X_1 being the 01/01/2024 and event_id_X_2 being 02/01/2024 (dd/mm/yyyy). 
We have excluded the actual dates so you can not go onto GEE to determine the value and that your model 
can be applied to any 2-year time period.
'''

# let's create a new a dataframe with columns for location, day, precipitation and flood label
train_ts['event_id'] = train_ts['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_ts['event_id'] = test_ts['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))

train_ts['event_t'] = train_ts.groupby('event_id').cumcount()
test_ts['event_t'] = test_ts.groupby('event_id').cumcount()

print(train_ts.head(n=10))
print(test_ts.head(n=10))

          event_id  precipitation  label  event_t
0  id_spictby0jfsb       0.000000      0        0
1  id_spictby0jfsb       0.095438      0        1
2  id_spictby0jfsb       1.949560      0        2
3  id_spictby0jfsb       3.232160      0        3
4  id_spictby0jfsb       0.000000      0        4
5  id_spictby0jfsb      56.025400      0        5
6  id_spictby0jfsb       1.097630      0        6
7  id_spictby0jfsb      23.265700      0        7
8  id_spictby0jfsb       2.521400      0        8
9  id_spictby0jfsb       0.000000      0        9
          event_id  precipitation  event_t
0  id_j7b6sokflo4k        0.00000        0
1  id_j7b6sokflo4k        3.01864        1
2  id_j7b6sokflo4k        0.00000        2
3  id_j7b6sokflo4k       16.61520        3
4  id_j7b6sokflo4k        2.56706        4
5  id_j7b6sokflo4k        0.00000        5
6  id_j7b6sokflo4k        0.00000        6
7  id_j7b6sokflo4k        0.00000        7
8  id_j7b6sokflo4k        0.00000        8
9  id_j7b6sokflo4k  

In [6]:
# let's start by seeing how many events are labeled as floods
total_events = len(train_ts)
num_floods = len(train_ts.query('label == 1'))
print(f'Total number of events labeled as floods: {num_floods}/{total_events} ({(num_floods/total_events)*100.0})')

Total number of events labeled as floods: 318/492020 (0.0646315190439413)


In [7]:
# under 0.5% of the data are floods, so let's see how many locations there are
location_ids = np.unique(train_ts['event_id'])
print(f'Total locations tracked: {len(location_ids)}')

Total locations tracked: 674


In [8]:
# let's see which locations experience a flood

floods_per_location = {} # maps a location_id to the number of floods it has
for i, location_id in enumerate(location_ids):
    if i % 50 == 0:
        print(f'Getting floods for location {location_id}...({i}/{len(location_ids)})')

    # get all the events for the location
    all_events = train_ts[train_ts.event_id == location_id]

    # get all flood events
    flood_events = all_events[all_events.label == 1]

    floods_per_location[location_id] = len(flood_events)

floods_per_location_df = pd.DataFrame.from_dict({'locations':list(floods_per_location.keys()), 'flood':list(floods_per_location.values())})


Getting floods for location id_05v6zjuaf300...(0/674)
Getting floods for location id_33zbia3ufza7...(50/674)
Getting floods for location id_6dsap350hxxn...(100/674)
Getting floods for location id_8y98mw911jlz...(150/674)
Getting floods for location id_bmbwxbwe99xl...(200/674)
Getting floods for location id_e9tnifct8ze6...(250/674)
Getting floods for location id_h11w3q9k2hbr...(300/674)
Getting floods for location id_jev5m3ynwb96...(350/674)
Getting floods for location id_mfuskvdk73tt...(400/674)
Getting floods for location id_p5ubwgtzf56t...(450/674)
Getting floods for location id_rf31s1g9mjox...(500/674)
Getting floods for location id_towbn6cp42pw...(550/674)
Getting floods for location id_waydwtcrhvi7...(600/674)
Getting floods for location id_yll130iyg7fk...(650/674)


In [9]:
# around 47% of locations experience a flood event, so a little under half

print(f'Floods per location: \n{floods_per_location_df.head(n=10)}')

flood_locations = floods_per_location_df.query('flood == 1')
print(f'Locations that experienced a flood: {len(flood_locations)}/{len(location_ids)} ({(len(flood_locations)/len(location_ids))*100.0})')


Floods per location: 
         locations  flood
0  id_05v6zjuaf300      1
1  id_06zma02zeea7      0
2  id_08w2po0cz63y      0
3  id_092vetuky9ku      0
4  id_0987b1h04r48      1
5  id_0b8wtfxfg90r      1
6  id_0cnfjlvz0zup      0
7  id_0cpm4w3t78ic      1
8  id_0ees839cilxs      0
9  id_0f27uesmwco5      0
Locations that experienced a flood: 318/674 (47.18100890207715)


In [33]:
'''
We have 2 years of data for each location, where event_t=0 means 01/01/yy, event_t=1 means 01/02/yy, and so on
Where dates are in dd/mm/yy format
Since there's 730 days per location, we can safely assume no leap years (I think - otherwise there'd be 731 days)
'''

# let's see how much rain each month had during its 2-year window across all locations
MONTHS = {
    'january':{'y1_start':0, 'y1_end':30, 'y2_start':365, 'y2_end':395},
    'february':{'y1_start':31, 'y1_end':58, 'y2_start':396, 'y2_end':423},
    'march':{'y1_start':59, 'y1_end':89, 'y2_start':424, 'y2_end':454},
    'april':{'y1_start':90, 'y1_end':119, 'y2_start':455, 'y2_end':484},
    'may':{'y1_start':120, 'y1_end':150, 'y2_start':485, 'y2_end':515},
    'june':{'y1_start':151, 'y1_end':180, 'y2_start':516, 'y2_end':545},
    'july':{'y1_start':181, 'y1_end':211, 'y2_start':546, 'y2_end':576},
    'august':{'y1_start':212, 'y1_end':242, 'y2_start':577, 'y2_end':607},
    'september':{'y1_start':243, 'y1_end':272, 'y2_start':608, 'y2_end':637},
    'october':{'y1_start':273, 'y1_end':303, 'y2_start':638, 'y2_end':668},
    'november':{'y1_start':304, 'y1_end':333, 'y2_start':669, 'y2_end':698},
    'december':{'y1_start':334, 'y1_end':364, 'y2_start':699, 'y2_end':729}
    }

rainfall_per_month = {
    'month': [month for month in MONTHS], 
    'y1_rainfall':[0.0 for _ in range(len(MONTHS))], 
    'y2_rainfall':[0.0 for _ in range(len(MONTHS))],
    }
for i, month in enumerate(MONTHS):
    desc = f'Getting total rainfall across all locations for month of {month}...'
    for j, location in tqdm(enumerate(location_ids), desc=desc):
        y1_stmt = f'event_id == "{location_id}" & event_t >= {MONTHS[month]['y1_start']} & event_t <= {MONTHS[month]['y1_end']}'
        y2_stmt = f'event_id == "{location_id}" & event_t >= {MONTHS[month]['y2_start']} & event_t <= {MONTHS[month]['y2_end']}'

        # get all events for the location during month of y1
        y1_events = train_ts.query(y1_stmt)

        # get all events for the location during month of y2
        y2_events = train_ts.query(y2_stmt)

        # calculate total rainfall for location during month of y1
        rainfall_per_month['y1_rainfall'][i] += y1_events.precipitation.sum()

        # calculate total rainfall for location during month of y2
        rainfall_per_month['y2_rainfall'][i] += y2_events.precipitation.sum()

rainfall_per_month_df = pd.DataFrame.from_dict(rainfall_per_month)
print(rainfall_per_month_df)
rainfall_per_month_df.to_csv('rainfall_per_month.csv', index=False)


Getting total rainfall across all locations for month of january...: 674it [00:24, 27.39it/s]
Getting total rainfall across all locations for month of february...: 674it [00:24, 27.40it/s]
Getting total rainfall across all locations for month of march...: 674it [00:25, 26.27it/s]
Getting total rainfall across all locations for month of april...: 674it [00:25, 26.21it/s]
Getting total rainfall across all locations for month of may...: 674it [00:25, 26.22it/s]
Getting total rainfall across all locations for month of june...: 674it [00:25, 26.71it/s]
Getting total rainfall across all locations for month of july...: 674it [00:25, 26.54it/s]
Getting total rainfall across all locations for month of august...: 674it [00:25, 26.26it/s]
Getting total rainfall across all locations for month of september...: 674it [00:26, 25.71it/s]
Getting total rainfall across all locations for month of october...: 674it [00:25, 26.16it/s]
Getting total rainfall across all locations for month of november...: 67

        month   y1_rainfall    y2_rainfall
0     january  62941.128669  108286.462992
1    february  45450.975668  101887.085405
2       march  97384.593265   79513.842440
3       april  24676.744120   79396.856260
4         may  31164.324919   81300.158120
5        june  24603.930080    9298.526242
6        july  25217.929050   25730.747935
7      august   3436.868888     826.290300
8   september  16316.867483   31296.658214
9     october   4606.646977   23589.191200
10   november  60226.732580   35400.932686
11   december  35320.808038   89057.679744


In [ ]:
# let's see which months had the most rainfall for y1 and y2

In [ ]:
# let's see which months have the most flood events across all locations

In [ ]:
# let's see which locations experience the most rainfall across all months for y1 and y2

In [ ]:
# let's see how much rain each location gets for each month throughout y1 and y2